In [15]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import cv2
import os
import random
import numpy as np

In [2]:
# FOR MAGES WITHOUT A PERSON
def scrape_images(search_terms, site_searches=[]):
    print("SEARCHING FOR: "+str(search_terms))
    photo_list = []
    
    for search_term in search_terms:
        if 'adobe' in site_searches or 'all' in site_searches:
            try:
                page = requests.get("https://stock.adobe.com/search?filters%5Bcontent_type%3Aphoto%5D=1&filters%5Bcontent_type%3Aillustration%5D=1&filters%5Bcontent_type%3Azip_vector%5D=1&filters%5Bcontent_type%3Avideo%5D=1&filters%5Bcontent_type%3Atemplate%5D=1&filters%5Bcontent_type%3A3d%5D=1&filters%5Bcontent_type%3Aimage%5D=1&filters%5Borientation%5D=square&filters%5Breleases%3Ais_exclude%5D=1&filters%5Bisolated%5D=only&k="+search_term+"&order=relevance&safe_search=1&search_type=filter-select&limit=100&search_page=1&get_facets=1").text
                soup = BeautifulSoup(page, 'lxml')
                photos = soup.find("div", {"id": "search-results"}).find_all("div",{"class": "search-result-cell"})
                for photo in photos:
                    if not "spacer" in photo.find("div", {"class": "thumb-frame"}).find("a").find("img")["src"]:
                        photo_list.append(photo.find("div", {"class": "thumb-frame"}).find("a").find("img")["src"])        
            except Exception as e:
                print("ADOBE: "+str(e))

        if 'istock' in site_searches or 'all' in site_searches:
            try:
                page = requests.get("https://www.istockphoto.com/photos/"+search_term+"?numberofpeople=none&orientations=square&phrase="+search_term+"&sort=mostpopular").text
                soup = BeautifulSoup(page, 'lxml')
                photos = soup.find("div", {"class": "search-content__gallery-assets"}).find_all("gi-asset")
                for photo in photos:
                    photo_list.append(photo.find("article").find("a").find("figure").find("img")["src"])        
            except Exception as e:
                print("ISTOCK: "+str(e))
        if 'getty' in site_searches or 'all' in site_searches:        
            try:
                page = requests.get("https://www.gettyimages.com/photos/"+search_term+"?license=rf&compositions=portrait,closeup,stilllife&family=creative&numberofpeople=none&orientations=square&phrase="+search_term+"&sort=mostpopular#license").text
                soup = BeautifulSoup(page, 'lxml')
                photos = soup.find("div", {"class": "search-content__gallery-assets"}).find_all("gi-asset", {"class": "gallery-mosaic-asset"})
                for photo in photos:
                    photo_list.append(photo.find("article").find("a").find("figure").find("img")["src"])
            except Exception as e:
                print("GETTY: "+str(e))

        if 'flickr' in site_searches or 'all' in site_searches:
            try:
                page = requests.get("https://www.flickr.com/search/?media=photos&text="+search_term+"&orientation=square").text
                soup = BeautifulSoup(page, 'lxml')
                photos_container = soup.find("div", {"class": "photo-list-view"})
                if photos_container:
                    photos = photos_container.find_all("div", {"class": "photo-list-photo-view"})
                    for photo in photos:
                        style = photo['style']
                        style_list = list(style.split(";")) 
                        photo_list.append("http:" + style_list[5].split(":")[1].split("(")[1].split(")")[0])
            except Exception as e:
                print("FLICKR: "+str(e))
                
        if 'google' in site_searches or 'all' in site_searches:
            try:
                page = requests.get("https://www.google.com/search?tbm=isch&q="+search_term+"&sa=X&biw=1400&bih=620&dpr=2").text
                soup = BeautifulSoup(page, 'lxml')
                photos = soup.find_all("img", {"class":"t0fcAb"})
                for photo in photos:
                    photo_list.append(photo['src'])
            except Exception as e:
                print("GOOGLE: "+str(e))
        

    return photo_list

In [49]:
# # FOR IMAGES OF A PERSON
# def scrape_images_person(search_terms, site_searches=[]):
#     photo_list = []
    
#     for search_term in search_terms:
#         if 'adobe' in site_searches or 'all' in site_searches:            
#             try:
#                 for x in range(1,101):            
#                     page = requests.get("https://stock.adobe.com/search?filters%5Bcontent_type%3Aphoto%5D=1&filters%5Bcontent_type%3Aillustration%5D=1&filters%5Bcontent_type%3Azip_vector%5D=1&filters%5Bcontent_type%3Avideo%5D=1&filters%5Bcontent_type%3Atemplate%5D=1&filters%5Bcontent_type%3A3d%5D=1&filters%5Bcontent_type%3Aimage%5D=1&filters%5Breleases%3Ais_include%5D=1&filters%5Borientation%5D=square&filters%5Bisolated%5D=only&k="+search_term+"&order=relevance&safe_search=1&search_type=filter-select&limit=100&search_page="+str(x)+"&get_facets=1").text
#                     soup = BeautifulSoup(page, 'lxml')
#                     photos = soup.find("div", {"id": "search-results"}).find_all("div",{"class": "search-result-cell"})
#                     for photo in photos:
#                         if not "spacer" in photo.find("div", {"class": "thumb-frame"}).find("a").find("img")["src"]:
#                             photo_list.append(photo.find("div", {"class": "thumb-frame"}).find("a").find("img")["src"])        
#             except Exception as e:
#                 print("ADOBE: "+str(e))
        
#         if "istock" in site_searches or 'all' in site_searches:
#             try:
#                 for x in range(1,101):            
#                     page = requests.get("https://www.istockphoto.com/photos/"+search_term+"?numberofpeople=one&page="+str(x)+"&orientations=square&phrase="+search_term+"&sort=mostpopular").text
#                     soup = BeautifulSoup(page, 'lxml')
#                     photos = soup.find("div", {"class": "search-content__gallery-assets"}).find_all("gi-asset")
#                     for photo in photos:
#                         photo_list.append(photo.find("article").find("a").find("figure").find("img")["src"])        
#             except Exception as e:
#                 print("ISTOCK: "+str(e))
                
#         if "getty" in site_searches or 'all' in site_searches:        
#             try:
#                 for x in range(1,101):
#                     page = requests.get("https://www.gettyimages.com/photos/"+search_term+"?license=rf&compositions=portrait,closeup,stilllife&family=creative&page="+str(x)+"&numberofpeople=1&orientations=square&phrase="+search_term+"&sort=mostpopular#license").text
#                     soup = BeautifulSoup(page, 'lxml')
#                     photos = soup.find("div", {"class": "search-content__gallery-assets"}).find_all("gi-asset", {"class": "gallery-mosaic-asset"})
#                     for photo in photos:
#                         photo_list.append(photo.find("article").find("a").find("figure").find("img")["src"])
#             except Exception as e:
#                 print("GETTY: "+str(e))
                
#         if "flickr" in site_searches or 'all' in site_searches:
#             try:
#                 page = requests.get("https://www.flickr.com/search/?media=photos&text="+search_term+"&orientation=square").text
#                 soup = BeautifulSoup(page, 'lxml')
#                 photos_container = soup.find("div", {"class": "photo-list-view"})
#                 if photos_container:
#                     photos = photos_container.find_all("div", {"class": "photo-list-photo-view"})
#                     for photo in photos:
#                         style = photo['style']
#                         style_list = list(style.split(";")) 
#                         photo_list.append("http:" + style_list[5].split(":")[1].split("(")[1].split(")")[0])
#             except Exception as e:
#                 print("FLICKR: "+str(e))
        
#         if "google" in site_searches or 'all' in site_searches:        
#             try:
#                 page = requests.get("https://www.google.com/search?tbm=isch&q="+search_term+"&sa=X&biw=1400&bih=620&dpr=2").text
#                 soup = BeautifulSoup(page, 'lxml')
#                 photo_rows = soup.find_all("td")
#                 for photo_row in photo_rows:
#                     image_url = photo_row.find('a').find('img')['src']
#                     photo_list.append(image_url)
#             except Exception as e:
#                 print("GOOGLE: "+str(e))
#     print(len(photo_list))

#     return photo_list

In [5]:
search_terms = ["pineapple","pineapple+fruit","pineappe+plant", "pineapple+plantation", "pineapple+field", "pineapple+fruit+stand", "pineapple+fruit+store"]

# search_terms = ["asian+male+profile","african+american+male+profile","white+male+profile","european+male+profile","indian+male+profile",
#                    "latino+male+profile","english+male+profile","black+male+profile","french+male+profile","young+male+profile","older+male+profile"]

# search_terms = ["african+american+woman+studio+shot","african+american+woman+close+up","african+american+female+studio+shot","african+american+female+face","african+american+woman+face",
#                   "african+american+woman+portait","african+american+woman+head+and+shoulders","african+american+woman+face+isolated",
#                   "african+american+woman+smiling+close+up"+"african+american+female+frowning+close+up","close-up+of+african+american+woman","close-up+of+african+american+female",
#                   "african+american+woman+laughing+close+up","african+american+woman+angry+close+up","african+american+woman+single+close+up",
#                   "african+american+woman+open+mouth+close+up","african+american+woman+with+glasses+close+up","african+american+woman+head+shoulders+close+up","african+american+woman+drinking+close+up",
#                   "african+american+woman+crying+close+up","african+american+woman+surprised+close+up","african+american+woman+calm+close+up","african+american+female+concerned+close+up",
#                   "african+american+female+talking+close+up","african+american+woman+shouting+close+up","african+american+female+young+close+up","african+american+female+elderly+close+up",
#                   "african+american+female+black+close+up","african+american+female+white+close+up","african+american+female+close+up","african+american+female+close+up",
#                   "african+american+female+european+close+up","african+american+female+african+american+close+up","african+american+female+middle+age+close+up",
#                   "african+american+female+thin+studio+shot","african+american+female+heavy+studio+shot","african+american+female+bald+close+up"]


In [6]:
image_list = scrape_images(search_terms, ["all"])

SEARCHING FOR: ['pineapple', 'pineapple+fruit', 'pineappe+plant', 'pineapple+plantation', 'pineapple+field', 'pineapple+fruit+stand', 'pineapple+fruit+store']


In [7]:
print(len(image_list))

460


In [37]:
def store_raw_images(image_url_list, directory):
    pic_num = 1    
    for image_url in image_url_list:
        try:
            urllib.request.urlretrieve(image_url, directory+"pineapple"+str(pic_num)+".jpg")
            pic_num += 1
        
        except Exception as e:
            print(str(e))

In [ ]:
store_raw_images(image_list, "dataset/images/"):

In [34]:
def augment_raw_images(directory):
    pic_num = 1
    for base_img in os.listdir(directory):
        pic_num += 1
        try:
            img = cv2.imread(directory+"/"+base_img)
            angle = int(random.uniform(-30, 30))
            h, w = img.shape[:2]
            M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
            img = cv2.warpAffine(img, M, (w, h))
            cv2.imwrite(directory+'/pineapple_aug'+str(pic_num)+".jpg", img)
        except Exception as e:
            print(str(e))

In [36]:
augment_raw_images("dataset/images/")

'NoneType' object has no attribute 'shape'
